In [ ]:
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes triton
!pip install xformers --index-url https://download.pytorch.org/whl/cu121

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-t2ia1p4g/unsloth_7af9f867dc8e4ff08916a9f8595168da
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-t2ia1p4g/unsloth_7af9f867dc8e4ff08916a9f8595168da
  Resolved https://github.com/unslothai/unsloth.git to commit 0fbbdfc091fc1a3b1c09b752794963681d10fad2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 90.7 MB/s eta 0:00:00
     ━━━━━━

In [ ]:
from unsloth import FastLanguageModel
import torch,os
max_seq_length = 2048
dtype = None
load_in_4bit = True
os.environ["HF_TOKEN"] = "your_Token"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
prompt = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. Dont add \n characters.
Do not include "SELECT short\_name, long\_name" this type of queries which have backslash in them.
You must output the SQL query that answers the question in a single line.

### Question:
{}

### Context:
{}

### Answer:
{}
"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    questions = examples["question"]
    contexts       = examples["context"]
    answers      = examples["answer"]
    texts = []
    for question, context, answer in zip(questions, contexts, answers):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(question, context, answer) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("b-mc2/sql-create-context", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset

Dataset({
    features: ['answer', 'question', 'context', 'text'],
    num_rows: 78577
})

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/78577 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 78,577 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.937600
2,3.022400
3,2.556300
4,2.211400
5,1.787700
6,1.477000
7,1.079300
8,0.935500
9,0.686500
10,0.661900


In [ ]:
model.save_pretrained("sql_tune") # Local saving
tokenizer.save_pretrained("sql_tune")

('sql_tune/tokenizer_config.json',
 'sql_tune/special_tokens_map.json',
 'sql_tune/tokenizer.json')

In [ ]:
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
from langchain_community.utilities.sql_database import SQLDatabase
db = SQLDatabase.from_uri(f"sqlite:///chinook.db")
db.get_table_info()

'\nCREATE TABLE albums (\n\t"AlbumId" INTEGER NOT NULL, \n\t"Title" NVARCHAR(160) NOT NULL, \n\t"ArtistId" INTEGER NOT NULL, \n\tPRIMARY KEY ("AlbumId"), \n\tFOREIGN KEY("ArtistId") REFERENCES artists ("ArtistId")\n)\n\n/*\n3 rows from albums table:\nAlbumId\tTitle\tArtistId\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/\n\n\nCREATE TABLE artists (\n\t"ArtistId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(120), \n\tPRIMARY KEY ("ArtistId")\n)\n\n/*\n3 rows from artists table:\nArtistId\tName\n1\tAC/DC\n2\tAccept\n3\tAerosmith\n*/\n\n\nCREATE TABLE customers (\n\t"CustomerId" INTEGER NOT NULL, \n\t"FirstName" NVARCHAR(40) NOT NULL, \n\t"LastName" NVARCHAR(20) NOT NULL, \n\t"Company" NVARCHAR(80), \n\t"Address" NVARCHAR(70), \n\t"City" NVARCHAR(40), \n\t"State" NVARCHAR(40), \n\t"Country" NVARCHAR(40), \n\t"PostalCode" NVARCHAR(10), \n\t"Phone" NVARCHAR(24), \n\t"Fax" NVARCHAR(24), \n\t"Email" NVARCHAR(60) NOT NULL, \n\t"SupportRepId" INTEGER

In [ ]:
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "sql_tune",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit
# )
FastLanguageModel.for_inference(model)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

In [ ]:
def find_relevant_tables_and_columns(question: str) -> str:
    """Search the database schema for relevant tables and columns based on the question."""
    table_info = db.get_table_info()  # Returns a string with table schemas
    relevant_tables = []

    # Split the schema info into individual tables
    tables = table_info.split("Table: ")

    for table in tables:
        # Check if any keyword from the question is present in the table schema
        if any(keyword.lower() in table.lower() for keyword in question.split()):
            relevant_tables.append(f"Table: {table}")

    # Return the schema for relevant tables, or an appropriate message if none are found
    if relevant_tables:
        context = "\n".join(relevant_tables)
    else:
        context = "No specific table found related to the question."
    return context

In [ ]:
def query_with_context(question: str):
    # Find the relevant tables and their context
    context = find_relevant_tables_and_columns(question)

    # Format the prompt
    prompt = f"""
    You are a powerful text-to-SQL model. Your job is to answer questions about a database.
    You are given a question and context regarding one or more tables.
    Do not include any backslash characters in your query.

    ### Question:
    {question}

    ### Context:
    {context}

    ### Answer:
    """

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=100)

    # Decode the generated query
    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Answer:")[1].strip()
    print(sql_query)
    try:
      result = db.run(sql_query)
      status = True
    except Exception as e:
      result = str(e)
      status = False
    return result, status

In [ ]:
question = "What are the top 10 tracks by play count?"
result = ""
for _ in range(0,5):
  result, status = query_with_context(question)
  if status: break
result

SELECT COUNT(track_id) FROM tracks ORDER BY track_id DESC LIMIT 10
SELECT T2.Name, COUNT(*) FROM tracks AS T1 JOIN customers AS T2 ON T1.TrackId = T2.CustomerId GROUP BY T1.TrackId ORDER BY COUNT(*) DESC LIMIT 10
SELECT T3.Name FROM tracks AS T1 JOIN invoice_items AS T2 ON T1.TrackId = T2.TrackId JOIN customers AS T3 ON T2.InvoiceId = T3.CustomerId GROUP BY T2.InvoiceId ORDER BY COUNT(T2.InvoiceId) DESC LIMIT 10
SELECT T2.Name, COUNT(*) FROM tracks AS T1 JOIN genres AS T2 ON T1.GenreId = T2.GenreId GROUP BY T2.Name ORDER BY COUNT(*) DESC LIMIT 10


"[('Rock', 1297), ('Latin', 579), ('Metal', 374), ('Alternative & Punk', 332), ('Jazz', 130), ('TV Shows', 93), ('Blues', 81), ('Classical', 74), ('Drama', 64), ('R&B/Soul', 61)]"